引入套件先

In [1]:
import cv2
import mediapipe as mp
import time
import math

手勢辨識

In [28]:
def vector_2d_angle(v1,v2): # 求出v1,v2兩條向量的夾角
    v1_x=v1[0]
    v1_y=v1[1]
    v2_x=v2[0]
    v2_y=v2[1]
    try:
        angle_= math.degrees(math.acos((v1_x*v2_x+v1_y*v2_y)/(((v1_x**2+v1_y**2)**0.5)*((v2_x**2+v2_y**2)**0.5))))
    except:
        angle_ = 100000.
    return angle_
    
def hand_angle(hand_):
    angle_list = []
    #---------------------------- thumb 大拇指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[2][0])),(int(hand_[0][1])-int(hand_[2][1]))),
        ((int(hand_[3][0])- int(hand_[4][0])),(int(hand_[3][1])- int(hand_[4][1])))
        )
    angle_list.append(angle_)
    #---------------------------- index 食指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])-int(hand_[6][0])),(int(hand_[0][1])- int(hand_[6][1]))),
        ((int(hand_[7][0])- int(hand_[8][0])),(int(hand_[7][1])- int(hand_[8][1])))
        )
    angle_list.append(angle_)
    #---------------------------- middle 中指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[10][0])),(int(hand_[0][1])- int(hand_[10][1]))),
        ((int(hand_[11][0])- int(hand_[12][0])),(int(hand_[11][1])- int(hand_[12][1])))
        )
    angle_list.append(angle_)
    #---------------------------- ring 無名指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[14][0])),(int(hand_[0][1])- int(hand_[14][1]))),
        ((int(hand_[15][0])- int(hand_[16][0])),(int(hand_[15][1])- int(hand_[16][1])))
        )
    angle_list.append(angle_)
    #---------------------------- pink 小拇指角度
    angle_ = vector_2d_angle(
        ((int(hand_[0][0])- int(hand_[18][0])),(int(hand_[0][1])- int(hand_[18][1]))),
        ((int(hand_[19][0])- int(hand_[20][0])),(int(hand_[19][1])- int(hand_[20][1])))
        )
    angle_list.append(angle_)
    return angle_list
    
def hand_gesture(angle_list):
    gesture_str = None
    if 100000. not in angle_list:
        if  (angle_list[1]<40) and (angle_list[2]<90 and angle_list[2]>20) and (angle_list[3]>40) and (angle_list[4]>40) :
            gesture_str = "無量空處"
    return gesture_str

偵測


In [ ]:
cap=cv2.VideoCapture(0,cv2.CAP_DSHOW)

mpHands=mp.solutions.hands
hands=mpHands.Hands()
mpDraw=mp.solutions.drawing_utils

pTime=0
cTime=0
while True:
  success, img = cap.read()
  imgRGB = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
  results = hands.process(imgRGB)
  # print(results.multi_hand_landmarks)
  if results.multi_hand_landmarks:
    z1=0
    z2=0
    x1=0
    x2=0
    for handLms in results.multi_hand_landmarks:
      hand_position=[]
      for id, lm in enumerate(handLms.landmark):
        h,w,c = img.shape
        cx, cy = int(lm.x*w),int(lm.y*h)
        hand_position.append((cx,cy))
        # if(id==8 or id==12):
        #   print(id,(cx,cy))
        if(id==8):
          x1=lm.x
          z1=lm.z
        if(id==12):
          x2=lm.x
          z2=lm.z  
        #putText 參數
        # img 原圖
        # text 要寫的文字
        # position 寫文字的位置
        # font 字體 (可以參考下圖)
        # size 字體大小
        # color 字的顏色
        # thickness 字的線條寬度
        # lineType 線條種類
        cv2.putText(img, str(int(id)),(cx+10,cy+10), cv2.FONT_HERSHEY_PLAIN,
              1,(0,0,255),2)
        #drawLandmark參數
        # 1、 image，需要畫圖的原始圖片
        # 2、 landmark_list 檢測到的人臉坐標
        # 3、 connections，連接線，需要把那些坐標連接起來
        # 4、 landmark_drawing_spec，坐標的顏色，粗細
        # 5、 connection_drawing_spec連接線的粗細，顏色等
      mpDraw.draw_landmarks(img,handLms,mpHands.HAND_CONNECTIONS)
      angleList=hand_angle(hand_position)
      gesture=hand_gesture(angleList)
      print((x1,x2))
      if(gesture=="無量空處" and abs(z1-z2)<0.04 and abs(x1-x2)<0.02) or (abs(x1-x2)<0.02):
        cv2.putText(img, "むりょうくうきょ",(w-200,70), cv2.FONT_HERSHEY_PLAIN,
              3,(255,0,255),2)
  cTime = time.time()
  fps = 1/(cTime-pTime)
  pTime=cTime
  cv2.putText(img, str(int(fps)),(10,70), cv2.FONT_HERSHEY_PLAIN,
              3,(255,0,255),3)
  cv2.imshow("image", img)
  if cv2.waitKey(1) & 0xFF == 27:
    cv2.destroyWindow('image')
    break
cap.release()